# Лекция 5: обработка ошибок, классы

На этой лекции рассмотрим обработку ошибок и взглянем на классы.

## Обработка ошибок  

### Типы ошибок 

* Основные идеи обработки ошибок в общем: коды возврата vs обработка исключений.
* В Python для обработки ошибок используются исключения.
* Два основных типа ошибок в Python: синтаксические ошибки и исключения.
* Синтаксические ошибки (SyntaxError) - ошибки синтаксиса языка, возникают во время парсинга файла с исходным кодом.
* Исключения (исключительные ситуации, exceptions) - ошибки возникающие во время выполнения программы.

### Исключения  

* При возникновении исключения создаётся объект-исключение, который также может содержать дополнительную информацию описывающую ошибку.  
* С места возникновения ошибки начинается "раскрутка стека": программа принудительно возвращается по иерархии вызовов функций до корня.  
* Если на этом "пути" исключение никак не обработано, то ошибка приводит к завершению программы.  
* Обработка исключений осуществляется с помощью блока try...except.
* Исключения могут возникать явно (оператор raise явно использован в пользовательском коде) и неявно (действия приводящие к ошибке, в т.ч. в сторонних библиотеках).  
* В стандартной библиотеке языка определён список встроенных типов исключений.  
* Есть возможность создания пользовательских типов исключений.

Рассмотрим простой пример обработки исключения при обращении к несуществующему ключу в словаре:

In [1]:
d = dict()

try:
    print(d["a"])
except Exception as e:
    print(type(e), e)

<class 'KeyError'> 'a'


Можно перечислять несколько вариантов перехватываемых (обычно от более точных к менее):

In [2]:
d = dict()

try:
    print(d["a"])
except KeyError as e:
    print("I see key error")
except Exception as e:
    print(type(e), e)

I see key error


Опишем вспомогательные функции:

In [3]:
def f(index):
    index = int(index)
    
    if index != 0:
        index = index + "a"

    msg = "Don't do that. Instead of raising bare exceptions, use more specific error type."
    raise Exception(msg)

def catch_that(func, val):
    try:
        func(val)
    except ValueError:
        print("ValueError. I don't want variable e so I don't mention it")
    except Exception as e:
        print("Exception ", type(e))
    except:
        print("Something strange is going on here")

In [4]:
catch_that(f, "a")

ValueError. I don't want variable e so I don't mention it


In [5]:
catch_that(f, -1)

Exception  <class 'TypeError'>


In [6]:
catch_that(f, 0)

Exception  <class 'Exception'>


### Замечания про бросание исключений  
* Про встроенные исключения можно почитать здесь: https://docs.python.org/3.5/library/exceptions.html#bltin-exceptions    
* При выбрасывании исключений старайтесь использовать как можно более близкий по смыслу тип встроенного исключения.  
* При необходимости, реализовывайте свои типы исключений, которые наследуйте от подходящих встроенных.  

### Ещё про перехват

In [7]:
def catch_that_again(f):
    try:
        f()
    except (ValueError, TypeError):
        print("Hey! I know that guy!")
    except:
        print("Huh?")
    else:
        print("Everythin is ok")
    finally:
        print("Finally!")

In [8]:
def raise_them_all():
    raise ValueError()

catch_that_again(raise_them_all)

Hey! I know that guy!
Finally!


In [9]:
def say_hello():
    print("Hello")

catch_that_again(say_hello)

Hello
Everythin is ok
Finally!


In [10]:
def break_things():
    a =  1 / 0

catch_that_again(break_things)

Huh?
Finally!


### Замечания по перехвату
* finally позволяет определить действия, которые выполнятся при выходе из блока try..except по любой причине (успешное выполнение, исключение, return, break, continue).
* Внимательнее с "except:" - такую форму использовать не рекомендуется, т.к. она очень просто маскирует ошибки.  
* Иногда требуется перехватить исключение, выполнить какие-то действия, и снова отправить исключение это же в путь - в таком случае используйте оператор raise без аргументов.
* Если же в подобном случае указать явно тип исключения в raise при его перевыбрасывании, то потеряется информация (например, stacktrace).

Пример повторного выбрасывания:

In [11]:
def catch_and_show():
    try:
        raise NameError("What are you doing at my house!?")
    except:
        print("Who are you?")
        raise

In [12]:
import sys
import traceback

try:
    catch_and_show()
except NameError as e:
    print(e, type(e))
    print("Go away! I will call the police!")
    traceback.print_tb(sys.exc_info()[2])

Who are you?
What are you doing at my house!? <class 'NameError'>
Go away! I will call the police!


  File "<ipython-input-12-57d0cbe16679>", line 5, in <module>
    catch_and_show()
  File "<ipython-input-11-fb9a6f84f747>", line 3, in catch_and_show
    raise NameError("What are you doing at my house!?")


## Пример простого класса

Попробуем написать и рассмотреть пример простого класса.

In [13]:
class Aggregator: # [3.x], before (object) was here
    SOME_CLASS_VARIABLE = "ahaha" + "aba"

    def __init__(self):
        self.total_sum = 0
        self.elements_count = 0

    def add_value(self, value):
        self.total_sum += value
        self.elements_count += 1

    def get_average(self):
        return self.total_sum / self.elements_count

    def get_sum(self):
        return self.total_sum

In [14]:
print(Aggregator.SOME_CLASS_VARIABLE)
print(type(Aggregator))

ahahaaba
<class 'type'>


### Разговор про обработку ошибок в больших проектах
* Экстренное завершение vs возврат в состояние
* Трейс и контекст ошибки
* Логирование
* Графики
* Статистика
* Мониторинги